This file creates a hexagonal grid around the US and assigns both the fire data and weather data to polygons within that grid.
It then outputs a dataframe that has aggregate weather data and fire data for each hex.

In [1]:
#This function gives a hexagonal grid around a certain long, lat based on a specified ring size and resolution

import h3
import geopandas as gpd
import shapely.geometry

def get_hexagon_grid(latitude, longitude, resolution, ring_size):
    """
    Generate a hexagonal grid GeoDataFrame centered around a specified location.

    Parameters:
    - latitude (float): Latitude of the center point.
    - longitude (float): Longitude of the center point.
    - resolution (int): H3 resolution for hexagons.
    - ring_size (int): Number of rings to create around the center hexagon (this will define the radius).

    Returns:
    - hexagon_df (geopandas.GeoDataFrame): GeoDataFrame containing hexagons and their geometries.
    """

    # Get the H3 hexagon for the center point
    center_h3 = h3.latlng_to_cell(latitude, longitude, resolution)
    
    # Generate hexagons within a disk around the center (instead of just rings)
    hexagons = list(h3.grid_disk(center_h3, ring_size))  # All hexagons within the distance
    
    # Create a GeoDataFrame with hexagons and their corresponding geometries
    hexagon_geometries = [shapely.geometry.Polygon(h3.cell_to_boundary(hexagon)) for hexagon in hexagons]
    
    # Create the GeoDataFrame
    hexagon_df = gpd.GeoDataFrame({'Hexagon_ID': hexagons, 'geometry': hexagon_geometries})

    return hexagon_df


In [2]:
#This function creates the hexagonal 

import geopandas as gpd
from shapely.geometry import Point

#This is a sped up version of the previous function
def calculate_hexagon_ids(df, hexagon_df):
    """
    Generates hexagon ids based on the previously calculated hexagonal grid 

    Parameters:
    - df (pandas.DataFrame): The dataframe that we want to add hexagon ids to
    - hexagon_df (geopandas.GeoDataFrame): Dataframe containing h3 hexes

    Returns:
    - df: Dataframe with an additional 'Hexagon_ID' column
    """
    
    # Convert hotel DataFrame to GeoDataFrame
    gdf_points = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['Latitude'], df['Longitude']),  crs="EPSG:4326")

    if gdf_points.crs != hexagon_df.crs:
       gdf_points = gdf_points.to_crs(hexagon_df.crs)
    # Perform spatial join
    joined = gpd.sjoin(gdf_points, hexagon_df, how='left', predicate='intersects')
    print(joined)

    # Return only the original DataFrame with the new Hexagon_ID column
    df['Hexagon_ID'] = joined['Hexagon_ID'].values

    return df


In [3]:
# Creation of the hexagonal grid 
#The input lat and lng are where the grid is centered- this can be moved to any lat, long - for example, over Califonia (lng = -119.45, lat = 37.17) 
input_lng = -103
input_lat = 44

# Generate H3 hexagons at a specified resolution - a larger resolution means smaller cell size
resolution = 3

# Indicate the number of rings around the central hexagon - larger ring size means more area of the map will be covered
ring_size = 25

# Hexagon grid around a given area
hexagon_df = get_hexagon_grid(input_lat, input_lng, resolution, ring_size)

# Visualize the first rows of the GeoDataFrame
hexagon_df.head()

,Hexagon_ID,geometry
0,832616fffffffff,"POLYGON ((43.448 -102.287, 43.927 -101.769, 44..."
1,83278bfffffffff,"POLYGON ((44.083 -103.536, 44.565 -103.02, 45...."
2,8326a5fffffffff,"POLYGON ((43.037 -103.676, 43.526 -103.166, 44..."
3,832612fffffffff,"POLYGON ((42.396 -102.439, 42.883 -101.928, 43..."
4,832610fffffffff,"POLYGON ((42.797 -101.059, 43.274 -100.541, 43..."


In [4]:
import geopandas as gpd
from shapely.geometry import Polygon
gdf = hexagon_df

#H3 swaps x and y coordinates for some reason, so we flip them 
def swap_coords(geom):
    if geom.geom_type == "Polygon":
        return Polygon([(y, x) for x, y in geom.exterior.coords])
    elif geom.geom_type == "MultiPolygon":
        return geom.__class__([Polygon([(y, x) for x, y in poly.exterior.coords]) for poly in geom.geoms])
    return geom
 
# Apply the function to fix the coordinates
gdf["geometry"] = gdf["geometry"].apply(swap_coords)

# Load the US states GeoDataFrame
us_states = gpd.read_parquet("../data/clean/maps/cb_2023_us_states_500k.parquet").to_crs(epsg=4326)

# Perform spatial join to get only intersecting hexagons
intersecting_hexagons = gpd.sjoin(gdf, us_states, how="inner", predicate="intersects")

# Optionally drop extra columns (e.g., state info) added from the join
intersecting_hexagons = intersecting_hexagons[hexagon_df.columns]

#Dropping duplicates
intersecting_hexagons = intersecting_hexagons.drop_duplicates(subset="Hexagon_ID")



C:\Users\natha\AppData\Local\Temp\ipykernel_96616\469466938.py:19: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  intersecting_hexagons = gpd.sjoin(gdf, us_states, how="inner", predicate="intersects")


In [5]:
#This code is only used to visualize the hexagonal map - it's not actually needed
import folium
import shapely.geometry

#Flipping the coordinates for mapping
gdf = intersecting_hexagons
gdf["geometry"] = gdf["geometry"].apply(swap_coords)

m = folium.Map(location=[37.5, -119.5], zoom_start=10)

# Add each hexagon polygon to the map
for _, row in gdf.iterrows():
    # Extract coordinates from the polygon geometry
    polygon = row['geometry']
    
    # Track polygon coordinates of the outside of the polygon
    coordinates = [(lon, lat) for lon, lat in polygon.exterior.coords]
    
    # Add the polygon to the map
    folium.Polygon(
        locations=coordinates,
        color='blue',
        weight=2,
        fill=True,
        fill_opacity=0.4
    ).add_to(m)

#Output the map
m

In [ ]:
#Saving to a parquet

#Flipping the coordinates again for joining later
hexagon_df["geometry"] = hexagon_df["geometry"].apply(swap_coords)

hexagon_df = gdf
hexagon_df.to_parquet("../data/curated/hexagon_data.parquet")

In [6]:
#Importing the fire data
import pandas as pd
import pyarrow.parquet as pq

# Reading in CSV - change path
file_path = '../data/clean/fire_occurrence_point/Fire_Occurence.parquet'
fire = pq.read_table(file_path)

fire = fire.to_pandas()

#May have run this if dataframe contains Latitude and Longitude instead of geometry
#fire = fire.rename(columns={'LATDD83': 'Latitude', 'LONGDD83': 'Longitude'})

fire = fire.head(100)

#May not have to run this if dataframe already contains Latitude and Longitude
from shapely.wkb import loads
fire['geometry'] = fire['geometry'].apply(loads)

fire['Latitude'] = fire['geometry'].apply(lambda x: x.y)
fire['Longitude'] = fire['geometry'].apply(lambda x: x.x)

fire

,OBJECTID,SIZECLASS,TOTALACRES,STATCAUSE,LATDD83,LONGDD83,DISCOVERYDATE,STATE,YEAR,MONTH,geometry,Hexagon_ID,Latitude,Longitude
0,231055009,A,0.25,Camping,39.84611,-106.42778,2016-10-22,CO,2016.0,10.0,POINT (-106.42778 39.84611),832681fffffffff,39.84611,-106.42778
1,231055010,A,0.10,Camping,39.37278,-107.93556,2016-08-07,CO,2016.0,8.0,POINT (-107.93556 39.37278),83269cfffffffff,39.37278,-107.93556
2,231055013,A,0.10,Camping,40.01929,-107.31731,2020-10-14,CO,2020.0,10.0,POINT (-107.31731 40.01929),832683fffffffff,40.01929,-107.31731
3,231055015,A,0.10,Camping,39.73639,-106.53639,2003-10-20,CO,2003.0,10.0,POINT (-106.53639 39.73639),832683fffffffff,39.73639,-106.53639
4,231055016,A,0.10,Lightning,39.48889,-106.32389,2012-07-05,CO,2012.0,7.0,POINT (-106.32389 39.48889),832683fffffffff,39.48889,-106.32389
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,231055199,A,0.10,Equipment,37.13417,-106.35278,2005-04-18,CO,2005.0,4.0,POINT (-106.35278 37.13417),8348dbfffffffff,37.13417,-106.35278
96,231055201,B,1.00,Smoking,37.81583,-106.35667,2000-06-11,CO,2000.0,6.0,POINT (-106.35667 37.81583),83268afffffffff,37.81583,-106.35667
97,231055202,B,1.00,Lightning,38.38889,-106.11111,2000-07-04,CO,2000.0,7.0,POINT (-106.11111 38.38889),83268efffffffff,38.38889,-106.11111
98,231055204,B,1.50,Incendiary,37.64139,-106.61722,2000-07-14,CO,2000.0,7.0,POINT (-106.61722 37.64139),83268afffffffff,37.64139,-106.61722


In [10]:
#Importing hexagonal grid to a geodataaframe
hexagon_df = gpd.GeoDataFrame(
    hexagon_df,
    geometry='geometry',
    crs="EPSG:4326"  
)



In [13]:
#Calculating a hexagonal id for each fire
#fire = fire.drop(columns=['Hexagon_ID']) #Drop this if we want to run it again - it already exists after first run in the updated file

fire = calculate_hexagon_ids(fire, hexagon_df)
fire.head()

     OBJECTID SIZECLASS  TOTALACRES   STATCAUSE   LATDD83   LONGDD83  \
0   231055009         A        0.25     Camping  39.84611 -106.42778   
1   231055010         A        0.10     Camping  39.37278 -107.93556   
2   231055013         A        0.10     Camping  40.01929 -107.31731   
3   231055015         A        0.10     Camping  39.73639 -106.53639   
4   231055016         A        0.10   Lightning  39.48889 -106.32389   
..        ...       ...         ...         ...       ...        ...   
95  231055199         A        0.10   Equipment  37.13417 -106.35278   
96  231055201         B        1.00     Smoking  37.81583 -106.35667   
97  231055202         B        1.00   Lightning  38.38889 -106.11111   
98  231055204         B        1.50  Incendiary  37.64139 -106.61722   
99  231055205         B        0.50   Lightning  37.58306 -106.38861   

   DISCOVERYDATE STATE    YEAR  MONTH                     geometry  Latitude  \
0     2016-10-22    CO  2016.0   10.0  POINT (39.84611 

,OBJECTID,SIZECLASS,TOTALACRES,STATCAUSE,LATDD83,LONGDD83,DISCOVERYDATE,STATE,YEAR,MONTH,geometry,Latitude,Longitude,Hexagon_ID
0,231055009,A,0.25,Camping,39.84611,-106.42778,2016-10-22,CO,2016.0,10.0,POINT (-106.42778 39.84611),39.84611,-106.42778,832681fffffffff
1,231055010,A,0.10,Camping,39.37278,-107.93556,2016-08-07,CO,2016.0,8.0,POINT (-107.93556 39.37278),39.37278,-107.93556,83269cfffffffff
2,231055013,A,0.10,Camping,40.01929,-107.31731,2020-10-14,CO,2020.0,10.0,POINT (-107.31731 40.01929),40.01929,-107.31731,832683fffffffff
3,231055015,A,0.10,Camping,39.73639,-106.53639,2003-10-20,CO,2003.0,10.0,POINT (-106.53639 39.73639),39.73639,-106.53639,832683fffffffff
4,231055016,A,0.10,Lightning,39.48889,-106.32389,2012-07-05,CO,2012.0,7.0,POINT (-106.32389 39.48889),39.48889,-106.32389,832683fffffffff


In [ ]:
#Saving the fire data
#fire.to_parquet("../data/curated/hexagon_data.parquet")